
> **AutoGen > Notebooks > StateFlow**
https://microsoft.github.io/autogen/0.2/docs/notebooks/agentchat_groupchat_stateflow/


# StateFlow : Build Workflows through State-Oriented Actions

**`AutoGen`**은 LLM(대형 언어 모델), 도구 또는 사람에 의해 구동되는 대화형 에이전트를 제공하며, 이들은 자동화된 채팅을 통해 함께 작업을 수행할 수 있도록 구성할 수 있다. 이 notebook에서는 AutoGen 에이전트를 사용해 **state**, 즉 상태 중심의 관점에서 *Groupchat*으로 워크플로우를 구축하는 방법을 소개 한다.

! `autogen-agentchat` 이 설치 되어 있어야 한다.
```python
pip install autogen-agentchat~=0.2
```

설치에 대한 더 자세한 정보는 아래의 **Installation guide**를 참고

> **Installation guide**
https://microsoft.github.io/autogen/0.2/docs/installation/

## See your API Endpoint
`config_list_from_json` 함수는 환경 변수 또는 JSON 파일에서 구성 목록을 로드한다.


In [4]:
from config import settings

api_key = settings.openai_api_key.get_secret_value()

In [12]:
import autogen

config_list = autogen.config_list_from_json(
    "/Users/geonheekim/Desktop/python_interview/autoGen/notebooks/ config_list.json",
    filter_dict={
        "tags" : ["gpt-4o-mini"],
    }
)

In [13]:
config_list

[{'model': 'gpt-4o-mini', 'tags': ['gpt-4o-mini']}]

### A workflow for research

연구를 위한 워크플로우를 위하여 다음과 같은 에이전트를 정의한다. 작업을 전달하여 워크플로우를 시작하는 **Initalizer(초기화자)**, 인터넷에서 논문을 검색할 수 있는 코드를 작성하는 **Coder(코더)**, 코드를 실행하는 **Executor(실행자)**,논문을 읽고 요약을 작성하는 **Scientist(과학자)**

4개의 상태(`Init`, `Retrieve`, `Research`, `End`)를 갖는 간단한 연구 플로우를 정의하고, 각 상태에서는 서로 다른 에이전트를 호출하여 작업을 수행한다.

- **Init** : *Initializer*를 사용해 워크플로우를 시작
- **Retrieve** : 먼저 Coder를 호출해 코드를 작성하고, 그 다음 *Executor*를 호출하여 코드를 실행한다.
- **Research** : *Scientist*를 호출하여 논문을 읽고 요약을 작성한다.
- **End** : 워크플로우를 종료한다.

에이전트 간의 전환을 정의함으로써 speaker selection method(화자 선택 방법)을 사용자 정의하여 상태 중심 워크플로우를 손쉽게 구현할 수 있다. 

In [16]:
import tempfile
from autogen.coding import LocalCommandLineCodeExecutor
from config import settings

api_key = settings.openai_api_key.get_secret_value()

In [20]:
temp_dir = tempfile.TemporaryDirectory()
executor = LocalCommandLineCodeExecutor(
    timeout=10,
    work_dir=temp_dir.name)

config_list = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key
            }
        ]
}

gpt4o_mini_config = {
    "cache_seed" : False,
    "temperature" : 0,
   "config_list" : config_list["config_list"],
   "timeout" : 120,
   }


In [21]:
initializer = autogen.AssistantAgent(
    name="Init",
    code_execution_config=False,
)

coder = autogen.AssistantAgent(
    name="Retrieve_Action_1",
    llm_config=gpt4o_mini_config,
    system_message = """You are the Coder. Given a topic, write code to retrieve related papers from the arXiv API, print their title, authors, abstract, and link.
You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type.
The user can't modify your code. So do not suggest incomplete code which requires others to modify.
Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result.
Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again.
Suggest the full code instead of partial code or code changes.
If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""" 
)

executor = autogen.AssistantAgent(
    name="Retrieve_Action_2",
    system_message="Executor. Execute the code written by the Coder and report the result.",
    human_input_mode= "NEVER",
    code_execution_config= {"executor" : executor},
)

scientist = autogen.AssistantAgent(
    name="Research_Action_1",
    llm_config=gpt4o_mini_config,
    system_message="""You are the Scientist. 
Please categorize papers after seeing their abstracts printed and create a markdown table with Domain, Title, Authors, Summary and Link""",
)

In [23]:
def state_transition(last_speaker, groupchat):
    messages = groupchat.messages
    
    if last_speaker == initializer:
        return coder
    elif last_speaker is coder:
        return executor
    elif last_speaker is executor:
        if messages[-1]["content"] == "exitcode: 1":
            return coder
        else:
            return scientist
    elif last_speaker == "Scientist":
        return None
        

In [27]:
groupchat = autogen.GroupChat(
    agents=[initializer, coder, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method = state_transition,
)

manager = autogen.GroupChatManager(
    groupchat= groupchat,
    llm_config = gpt4o_mini_config,
)

In [28]:
chat_result = initializer.initiate_chat(
    manager, message="Topic: LLM applications papers from last week. Requirement: 5 -10 papers from different domains."
)

Init (to chat_manager):

Topic: LLM applications papers from last week. Requirement: 5 -10 papers from different domains.

--------------------------------------------------------------------------------

Next speaker: Retrieve_Action_1

Retrieve_Action_1 (to chat_manager):

```python
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

def fetch_arxiv_papers(topic, days=7, max_results=10):
    # Calculate the date range for the last week
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    # Format dates for arXiv query
    start_date_str = start_date.strftime('%Y%m%d')
    end_date_str = end_date.strftime('%Y%m%d')
    
    # Construct the arXiv API query
    query = f"search_query=all:{topic}&start=0&max_results={max_results}&sortBy=submittedDate&sortOrder=descending&dateRange={start_date_str}-{end_date_str}"
    url = f"http://export.arxiv.org/api/query?{query}"
    
    # Make the request to the arXiv API
 